In [40]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [42]:
# Copy the dataframe
df2 = data.copy()

In [43]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [44]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [45]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'Chargetime']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['HP']]

In [46]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='unadjusted')

In [47]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -31.157
Estimator:                    IV-2SLS   Adj. R-squared:                -33.322
No. Observations:                 334   F-statistic:                    5.2025
Date:                Wed, Mar 06 2024   P-value (F-stat)                0.9998
Time:                        21:29:25   Distribution:                 chi2(21)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.4926     12.774    -0.4300     0.6672     -30.530      19.544
Range          0.0319     0.1352     0.2357     0.8137     -0.2332      0.2970
Chargetime    -0.0447     0.0976    -0.4575     0.6473     -0.2360      0.1467
Segment_B      0.4359     5.8718     0.0742     0.9408     -11.073      11.944
Segment_C      4.3103     23.238     0.1855     0.8529     -41.236      49.856
Segment_D      3.4198     21.815     0.1568     0.8754     -39.337      46.177
Segment_E      10.890     71.823     0.1516     0.8795     -129.88      151.66
Segment_F      22.038     136.25     0.1617     0.8715     -245.02      289.09
Segment_M      5.1088     37.651     0.1357     0.8921     -68.687      78.904
Segment_J      1.6039     18.354     0.0874     0.9304     -34.369      37.576
Year_2014     -0.1645     8.4584    -0.0194     0.9845     -16.743      16.414
Year_2015     -0.2689     7.3632    -0.0365     0.9709     -14.701      14.163
Year_2016      2.1371     13.773     0.1552     0.8767     -24.858      29.132
Year_2017      2.3150     15.503     0.1493     0.8813     -28.071      32.701
Year_2018      2.1820     17.860     0.1222     0.9028     -32.823      37.186
Year_2019      2.0254     18.967     0.1068     0.9150     -35.149      39.200
Year_2020      2.1072     22.745     0.0926     0.9262     -42.472      46.686
Year_2021      0.9158     21.756     0.0421     0.9664     -41.725      43.557
Year_2022      1.7337     28.375     0.0611     0.9513     -53.881      57.348
Year_2023      2.2892     34.764     0.0658     0.9475     -65.847      70.425
China         -7.0216     35.235    -0.1993     0.8420     -76.082      62.039
Price      -3.628e-05     0.0002    -0.1798     0.8573     -0.0004      0.0004
==============================================================================

Endogenous: Price
Instruments: HP
Unadjusted Covariance (Homoskedastic)
Debiased: False
"""